In [1]:
import numpy as np
import pandas as pd
from keras import layers
import itertools
import datetime
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
Data = pd.read_csv('Data_training_and_val_set_CNvsAD.csv')

In [3]:
#dropping irrelevant feature
Data.drop(['RID'], axis=1, inplace = True)

In [4]:
x = Data.iloc[:, :-1].values
y = Data.iloc[:, -1].values

In [5]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [6]:
#splitting the data into train,test 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.30)

In [12]:
# Define Model Section

In [8]:
start_time = datetime.datetime.now()
#creating the model
def create_model():
    model = Sequential()
    model.add(layers.Dense(15, activation='relu', kernel_initializer='random_normal', input_shape = (xtrain.shape[1],)))
    model.add(layers.Dense(15, activation='relu'))#, kernel_initializer='random_normal'))
    #model.add(layers.Dense(15, activation='relu'))#, kernel_initializer='random_normal'))
    #model.add(layers.Dense(15, activation='relu'))
    #model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))#, kernel_initializer='random_normal'))#used to be number of columns in the dataset
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [9]:
# validating

In [17]:
num_epochs = 50
all_acc_histories = []
all_loss_histories = []

k = 4
num_val_samples = len(partial_train_data) // k
batch_size = max(1, len(partial_train_data) // 10)


In [18]:
for i in range(k):
    print('processing fold #', i)
    valdata = partial_train_data[i * num_val_samples: (i + 1) * num_val_samples]
    valtargets = partial_train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    traindata = np.concatenate(
        [partial_train_data[:i * num_val_samples],
         partial_train_data[(i + 1) * num_val_samples:]],
        axis=0)
    traintargets = np.concatenate(
        [partial_train_targets[:i * num_val_samples],
         partial_train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    model = create_model()
    history = model.fit(traindata, traintargets,
                        validation_data=(valdata, valtargets),
                        epochs=num_epochs,
                        batch_size=batch_size, verbose=1)
    acc_history = history.history['val_accuracy']
    all_acc_histories.append(acc_history)
    loss_history = history.history['val_loss']
    all_loss_histories.append(loss_history)

processing fold # 0
Epoch 1/50


ValueError: ignored